In [1]:
"""This code calculated to use a high res. significant wave height in a tiff.
Written by Taran Kalra, July 22, 2024"""

'This code calculated to use a high res. significant wave height in a tiff.\nWritten by Taran Kalra, July 22, 2024'

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat              
import rioxarray
import rasterio
from create_raster import make_swan_raster_StructuredGrid

In [3]:
# Grid specific inputs
PR = 0 
St_Croix = 1
if(St_Croix):
    gridsize = 10  # in meter
    gridx = 483 #j direction + 1 from swan file 
    gridy = 278 #i direction + 1 from swan file 
    xMin = 389822 #this is your x origin in state plane m (or whatever you used)
    yMin = 192378 #this is your y origin in state plane m (or whatever you used)
            
    EPSG = 32163 # EPSGS correspondign to USVI (ST.Croix )
    angle_degrees = 0.0  # Angle of the grid
if(PR):
    gridsize = 5  # in meter
    gridx = 1095 # direction + 1 from swan file 
    gridy = 1048 #i direction + 1 from swan file 
    xMin = 316520 #this is your x origin in state plane m (or whatever you used)
    yMin = 247622 #this is your y origin in state plane m (or whatever you used)
            
    EPSG = 32161 # EPSGS correspondign to PR
    angle_degrees = 41.6  # Angle of the grid
    
m_to_feet = 3.28084
varname = "Hsig"

In [4]:
#matfile = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\66113_Coral_Reefs_PR\SwanRuns\Restored_Reef\1perc\1perc_restored_outs.mat'
cases = ["1perc", "2perc", "5perc", "10perc", "20perc", "50perc"]
scene = "Restored" # "Restored"

if (scene == "Restored"):
    scenario = "Restored" # "Degraded"
    scenario1= "restored" # degraded"  
    scenario_out = "alt"  # "no_alt"
    gridfile = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Grid\restored_USVI_rasterio.tif'

else:
    scenario = "Degraded"
    scenario1 = "degraded"
    scenario_out = "no_alt"
    gridfile = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Grid\degraded_USVI_rasterio.tif'

#outfile  = 'test.tif'

# input file names
#directory = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs\'
#input_directory = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs'
#gridfile = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\66113_Coral_Reefs_PR\Lidar_shapes\combined_mosaic_PRReefs.tif'

input_directory = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs'
output_directory = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_final\Wave_height'


In [5]:
# Loop over the filenames in the directory
for i, case in enumerate(cases):
    matfile = f'{input_directory}\{scenario}_Reef\{case}\{case}_{scenario1}_outs.mat'
    outfile = f'{output_directory}\{case}_{scenario_out}.tif' 

    print(matfile)
    print(outfile) 
    
# Temporary filenames   
    outfile_coarse_raster = 'tiffs_back\output_raster_coarse_Hsig3.tif'
    outfile_refine_raster = 'tiffs_back\output_raster_refine_Hsig3.tif'    
    
    # Get the Hsig 
    make_swan_raster_StructuredGrid(outfile_coarse_raster, matfile, varname, EPSG, xMin, yMin, gridx, gridy, gridsize, angle_degrees)

    # Open this newly created tiff on the SWAN grid 
    coarse_data = rioxarray.open_rasterio(outfile_coarse_raster)

    # Grid file path specified 
    # this is the finer TIFF file 
    finer_xds = rioxarray.open_rasterio(gridfile) 

    # Interpolate wave height from SWAN grid to the topobathy tiff (merged tiff)
    ds_new = coarse_data.interp(y=finer_xds.y, x=finer_xds.x, method='nearest')
    ds_new = ds_new.squeeze()
    print("Converting meters to feet")
    print(np.nanmin(finer_xds.x.values))
    ds_new = m_to_feet*ds_new
    print(type(ds_new))
    print(np.nanmin(finer_xds.x.values))
    ds_new.rio.to_raster(outfile_refine_raster)


    # Grid file used to copy the profile 
    with rasterio.open(gridfile, 'r') as src:
        profile1 = src.profile.copy()

    #   the profile in the output raster1 and then add feet in units 
    with rasterio.open(outfile_refine_raster) as src: 
        profile = src.profile
        profile = profile1 
        array = src.read(1)
    # Update the profile to change the units from meters to feet    
    profile['units'] = 'Feet'

    # Save the updated array to a new TIFF file
    with rasterio.open(outfile, 'w+', **profile) as dst:
        dst.write(array, 1)
        dst.update_tags(units=profile['units'])

    del outfile_refine_raster, outfile_coarse_raster

C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs\Restored_Reef\1perc\1perc_restored_outs.mat
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_final\Wave_height\1perc_alt.tif
raster finished
Converting meters to feet
389822.4054892473
<class 'xarray.core.dataarray.DataArray'>
389822.4054892473
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs\Restored_Reef\2perc\2perc_restored_outs.mat
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_final\Wave_height\2perc_alt.tif
raster finished
Converting meters to feet
389822.4054892473
<class 'xarray.core.dataarray.DataArray'>
389822.4054892473
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs\Restored_Reef\5perc\5perc_restored_outs.mat
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_final\Wave_height\5perc_alt.tif
raster finished
Converting mete

In [6]:
fine_data = rioxarray.open_rasterio(outfile)
#fine_data.plot()
fine_data

<xarray.DataArray (band: 1, y: 2771, x: 4821)>
[13358991 values with dtype=float32]
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 3.898e+05 3.898e+05 ... 3.946e+05 3.946e+05
  * y            (y) float64 1.951e+05 1.951e+05 ... 1.924e+05 1.924e+05
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:  Area
    units:          metre
    _FillValue:     -999999.0
    scale_factor:   1.0
    add_offset:     0.0

In [7]:
print(np.nanmin(ds_new.values))
print(np.nanmax(ds_new.values))

0.0
1.7762403
